In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd 
pd.set_option('display.max_columns', None)

In [ ]:
import os 
import datetime
datetime.datetime.fromtimestamp(os.path.getmtime('ds.csv')).strftime('%Y-%m-%d %H:%M:%S')

In [25]:
import requests
def stock_spot_us():
    url = 'http://40.push2.eastmoney.com/api/qt/clist/get'
    params = {
        'pn': '1',
        'pz': '15000',
        'po': '1',
        'np': '1',
        'ut': 'bd1d9ddb04089700cf9c27f6f7426281',
        'fltt': '2',
        'invt': '2',
        'fid': 'f3',
        'fs': 'm:105,m:106,m:107',
        'fields': 'f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f26,f22,f33,f11,f62,f128,f136,f115,f152',
        '_': '1631107510188',
    }

    r = requests.get(url, params=params, timeout=10)
    data_json = r.json()
    temp_df = pd.DataFrame(data_json['data']['diff'])
    temp_df.columns = [
        "_",
        "最新价",
        "涨跌幅",
        "涨跌额",
        "_",
        "_",
        "_",
        "_",
        "_",
        "_",
        "_",
        "简称",
        "编码",
        "名称",
        "最高价",
        "最低价",
        "开盘价",
        "昨收价",
        "总市值",
        "_",
        "_",
        "_",
        "_",
        "_",
        "_",
        "_",
        "_",
        "市盈率",
        "_",
        "_",
        "_",
        "_",
        "_",
    ]
    temp_df.reset_index(inplace=True)
    temp_df["index"] = range(1, len(temp_df) + 1)
    temp_df.rename(columns={"index": "序号"}, inplace=True)
    temp_df["证券代码"] = temp_df["简称"]
    temp_df = temp_df[
        [
            "证券代码",
            "名称",
            "最新价",
            "涨跌额",
            "涨跌幅",
            "开盘价",
            "最高价",
            "最低价",
            "昨收价",
            "总市值",
            "市盈率",
#             "代码",
        ]
    ]
    temp_df["最新价"] = pd.to_numeric(temp_df["最新价"], errors="coerce")
    temp_df["涨跌额"] = pd.to_numeric(temp_df["涨跌额"], errors="coerce")
    temp_df["涨跌幅"] = pd.to_numeric(temp_df["涨跌幅"], errors="coerce")
    temp_df["开盘价"] = pd.to_numeric(temp_df["开盘价"], errors="coerce")
    temp_df["最高价"] = pd.to_numeric(temp_df["最高价"], errors="coerce")
    temp_df["最低价"] = pd.to_numeric(temp_df["最低价"], errors="coerce")
    temp_df["昨收价"] = pd.to_numeric(temp_df["昨收价"], errors="coerce")
    temp_df["总市值"] = pd.to_numeric(temp_df["总市值"], errors="coerce")
    temp_df["市盈率"] = pd.to_numeric(temp_df["市盈率"], errors="coerce")    
    return temp_df

In [26]:
df = stock_spot_us()

df = df[~df['涨跌幅'].isnull()]
df = df[~df['总市值'].isnull()]
df.sort_values('总市值',ascending = False).head(1000)

,证券代码,名称,最新价,涨跌额,涨跌幅,开盘价,最高价,最低价,昨收价,总市值,市盈率
8403,AAPL,苹果,184.92,-1.09,-0.59,186.73,186.99,184.27,186.01,2.908552e+12,30.84
10067,MSFT,微软,342.33,-5.77,-1.66,351.32,351.47,341.95,348.10,2.545390e+12,36.88
9681,AMZN,亚马逊,125.49,-1.62,-1.27,127.71,127.90,125.30,127.11,1.287572e+12,299.85
3299,NVDA,英伟达,426.92,0.39,0.09,434.50,437.21,426.61,426.53,1.055828e+12,220.29
9804,GOOG,谷歌-C,124.06,-1.73,-1.38,126.70,126.70,123.79,125.79,8.381494e+11,14.31
...,...,...,...,...,...,...,...,...,...,...,...
3385,FTI,TechnipFMC plc,14.72,0.01,0.07,14.73,14.86,14.59,14.71,6.499807e+09,-144.12
5721,AMKR,艾马克技术,26.39,-0.01,-0.04,26.66,26.66,26.12,26.40,6.482458e+09,10.12
1914,DOOO,BRP Inc,82.85,0.64,0.78,82.03,83.23,81.90,82.21,6.469042e+09,10.03
11255,ESTC,Elastic NV,66.39,-3.07,-4.42,69.21,69.51,66.07,69.46,6.465781e+09,-27.38


# 美股

In [3]:
import akshare as ak
import pandas as pd 
# pd.set_option('display.max_rows', None)

us_df = pd.read_csv('us_stock.csv', encoding = 'gbk')
stock_us_spot = stock_spot_us()
stock_us_spot[['交易所代码', '证券代码']] = stock_us_spot['代码'].str.split('.', expand = True)
# stock_us_spot['总市值'] = (pd.to_numeric(stock_us_spot['总市值'], errors="coerce")/100000000).round(1).fillna('') 
df = us_df.merge(stock_us_spot, on = '证券代码', how = 'inner')
df = df[~df['三级行业'].isnull()]
df = df[~df['涨跌幅'].isnull()]
df = df[~df['总市值'].isnull()]
df['总市值'] = (df['总市值']/100000000).round(2)

ModuleNotFoundError: No module named 'akshare'

In [18]:
import plotly.express as px

fig = px.treemap(df, 
                 path=['一级行业','二级行业','三级行业','证券名称'],  # 指定层次结构，每一个层次都应该是category型的变量
                 values='总市值', # 需要聚合的列名
                 color='涨跌幅', 
                 custom_data=['涨跌幅','证券代码','总市值','最新价'],
                 range_color = [-8, 8], # 色彩范围最大最小值
                 hover_data= {'涨跌幅':':.2',
                             '总市值':':.1f'}, # 鼠标悬浮显示数据的格式
                 color_continuous_scale=["seagreen",'lightgrey', "indianred"],
                 height = 900,
                #  width = 1600,
                 color_continuous_midpoint=0 , # 颜色变化中间值设置为增长率=0
                )
# fig.update_layout({
# 'plot_bgcolor': 'rgba(0, 0, 0, 0)',
# 'paper_bgcolor': 'rgba(0, 0, 0, 0)',
# })
fig.update_coloraxes(showscale=False)
fig.update_traces(textposition='middle center', 
                  textinfo='label', 
                  textfont = dict(color='white'),
                  texttemplate= "%{label}<br>%{customdata[0]:.2f}%<br>",
                  hovertemplate= "%{customdata[1]}<br>%{label}<br>%{customdata[3]:.2f}  (%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿") # 显示企业名称和市值，字体24

fig.show()